## Assignment 19: The goal of this assignment was to run multiple random forests using undersampling and create a model using a majority vote.

### The first step was to upload and clean the bank data.  After that I followed the prompts laid out in the assignment.

In [3]:
bank = read.csv('hmeq.csv')
bank$BAD = as.factor(bank$BAD)
names(bank)[1] = "target"

quick_clean <- function(df, p)
{
if(p == 1)
{
fixcol = function(y)
{if(is.numeric(y) == TRUE)
{y[is.na(y)] = mean(y, na.rm = TRUE)}
  else
  {
      levels = unique(y)
      y[is.na(y)] = levels[which.max(tabulate(match(y, levels)))]
  }
  return(y)
  }
df = lapply(df, fixcol)
return(data.frame(df))
}
else if (p==0)
{
na.omit(df)
}
  else
  {print("Argument 2 needs to be 0 or 1")}
}

bank = quick_clean(bank, 1)
sum(is.na(bank))

[1] 0

### 1.	Train a random forest using undersampling. Let rf1 to be the trained model

In [4]:
library(caret)
splitIndex <- createDataPartition(bank$target, p = .70, list = FALSE, times = 1)
train <- bank[ splitIndex,]
test <- bank[-splitIndex,]

library(ranger)
train0 = train[train$target == "0",]
n0 = nrow(train0)
train1 = train[train$target == "1",]
n1 = nrow(train1)
train00 = train0[sample(1:n0, n1),]
train_under = rbind(train1, train00)
rf1 = ranger(target ~., data = train_under)

### 2.	Use rf1 to produce the prediction for the test set.  Let c1 to be a column vector contain all the predictions. The values in c1 should only be 0 or 1.

In [5]:
c1 = predict(rf1, data = test)$predictions
cm=confusionMatrix(c1, test$target, positive = "1")
cm$overall['Accuracy']
cm$byClass['Balanced Accuracy']

Accuracy 
0.8740907

Balanced Accuracy 
        0.8834005

### 3.	Redo 1 and 2 to create c2, c3, c4,., c1000
### 4.	Merge columns c1, c2, c1000 in to a matrix mc. 

In [6]:
n = 1000
mc = NULL
for(i in 1:n)
{
train0 = train[train$target == "0",]
n0 = nrow(train0)
train1 = train[train$target == "1",]
n1 = nrow(train1)
train00 = train0[sample(1:n0, n1),]
train_under = rbind(train1, train00)
rf = ranger(target ~., data = train_under)
c  = predict(rf, data = test)$predictions
mc = matrix(data = c(mc, c),ncol = i)
}

mc = mc - 1

### 5.	Let cc be a row mode of mc.  The vector cc is the majority vote of 1000 predictions for the test set. 

In [7]:
cc = as.factor(c(round(rowMeans(mc))))

### 6.	Produce the confusion matrix based on cc and the true target. 

In [8]:
cm1000 = confusionMatrix(cc, test$target, positive = "1")
cm1000$overall['Accuracy']
cm1000$byClass['Balanced Accuracy']
cm1000

print("The 1000 run simulations produced a better accuracy and balanced accuracy than the single run")

Accuracy 
0.8836038

Balanced Accuracy 
          0.88512

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1263   40
         1  168  316
                                          
               Accuracy : 0.8836          
                 95% CI : (0.8678, 0.8981)
    No Information Rate : 0.8008          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.6786          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.8876          
            Specificity : 0.8826          
         Pos Pred Value : 0.6529          
         Neg Pred Value : 0.9693          
             Prevalence : 0.1992          
         Detection Rate : 0.1768          
   Detection Prevalence : 0.2708          
      Balanced Accuracy : 0.8851          
                                          
       'Positive' Class : 1               
                                          

[1] "The 1000 run simulations produced a better accuracy and balanced accuracy than the single run"
